In [49]:
%load_ext autoreload
%autoreload 2

In [52]:
from gensim.models import Doc2Vec as D2V
from gensim.models.doc2vec import TaggedDocument

from nltk import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import download as nltkDownload

import os
from os import listdir
from os.path import isfile, join

import db_helper

import re

PROJECT_PATH='/home/hmenn/Workspace/CSE496_Node2Vec_2018'

In [53]:
# download nltk sources if not downloaded yet
nltkDownload('stopwords')
nltkDownload('punkt')

[nltk_data] Downloading package stopwords to /home/hmenn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/hmenn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
db = DBHelper()
db.connect(user="root",passwd="Hasan5695*",db="cse496")
tweets = db.getTweets("denemeShort")
tweets.sort(key=lambda t: t.date)

In [59]:
tweets = db_helper.getTweetsFromJson(PROJECT_PATH+'/dataset/SocialMediaDataset/tweets.json')

In [60]:
tweets[0]

############ Tweet: ############ 
SendByScreenName:cakrr_ 
UserID:u1623755515 
TweetID:t763141405766250496 
InReplyToStatusID:t763141168771244032 
InReplyToUserID:u757176735158923264 
Date:2016-08-10 01:34:24 
Text:@Wesleyoloji10 OGLAK HAZİNE ARIYOR AQ:das:asd 

In [61]:
tokenizer = RegexpTokenizer(r'\w+')
stopword_set = set(stopwords.words("turkish"))

In [62]:
# clear tweets
for tweet in tweets:
    tweet.text = re.sub(r"RT(^|[^@\w])@(\w{1,15})\b:","",tweet.text)
print(tweets[2].text)

Konut kira/satışları düşünüldüğünde piyasa öylesine şişti ki.Örneğin İstanbul'da 1000 liranın altında ev kiralamanız nerdeyse mümkün değil!


In [66]:
dataset = tweets[0:100]
dataset += tweets[3000:3050]
dataset += tweets[5000:5100]
dataset += tweets[8000:8050]
len(dataset)

300

In [63]:
def nlp_clean(data):
    new_data = []
    for d in data:
        new_str = d.lower()
        dlist = tokenizer.tokenize(new_str)
        dlist = list(set(dlist).difference(stopword_set))
        new_data.append(dlist)
    return new_data

In [67]:
cleanData = nlp_clean([tweet.text for tweet in dataset])

In [69]:
taggedData = [TaggedDocument(words=word_tokenize(_d[0].lower()), tags=[str(_d[1])]) for i, _d in enumerate([(tweet.text, tweet.tweetID) for tweet in dataset])]

In [70]:
print(len(taggedData)), taggedData[1]

300


(None,
 TaggedDocument(words=["sn.cb'nımızın", 'çağrısıyla', '3', 'banka', 'konut', 'kredisi', 'faiz', 'oranlarını', "℅1'in", 'altına', 'çekti..bu', 'olumlu', 'bir', 'adım', '.', 'ancak', 'şişen', 'piyasa', 'nasıl', 'düşecek', '?'], tags=['t763577912696135680']))

In [71]:
%%time
max_epochs = 500

model = D2V(alpha=0.025, 
                min_alpha=0.025,
                min_count=1,
                dm =1)
  
model.build_vocab(taggedData)

for epoch in range(1, max_epochs+1):
    if epoch %100 == 0:
        print('iteration {0}'.format(epoch))
    model.train(taggedData,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
model.save(PROJECT_PATH + "/outputs/tweets_d2v.model")

/home/hmenn/Workspace/CSE496_Node2Vec_2018/n2vEnv/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  from ipykernel import kernelapp as app


iteration 100
iteration 200
iteration 300
iteration 400
iteration 500
CPU times: user 38.2 s, sys: 815 ms, total: 39 s
Wall time: 39 s


In [77]:
#model.docvecs.most_similar("t765296806276268032")
model.docvecs.similarity('t765466344707989504','t765582802343657472')

0.7528703228033607

In [80]:
for t in dataset:
    if t.tweetID=="t765466344707989504" or t.tweetID=="t765582802343657472":
        print(t)

############ Tweet: ############ 
SendByScreenName:demirmadeni 
UserID:u1658471437 
TweetID:t765466344707989504 
InReplyToStatusID:t-1 
InReplyToUserID:u-1 
Date:2016-08-16 11:32:52 
Text: Galatasaray basketbol şubesinden son haberler!
https://t.co/WQTFKf7X3D https://t.co/T7Ex0UiHgn 

############ Tweet: ############ 
SendByScreenName:demarkesports 
UserID:u1499266621 
TweetID:t765582802343657472 
InReplyToStatusID:t-1 
InReplyToUserID:u-1 
Date:2016-08-16 19:15:38 
Text:#Rio2016 Kadınlar Basketbol çeyrek final maçında Sırbistan, Avustralya'yı 73-71 mağlup etti ve yarı finale yükseldi. https://t.co/akxCoNG6nJ 



In [78]:
import gc
gc.collect()

146